In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers.trainer import TrainingArguments
from peft import LoraConfig, AutoPeftModelForCausalLM, get_peft_model, prepare_model_for_int8_training, PeftModel
from trl import SFTTrainer, setup_chat_format
from datasets import load_dataset
from transformers import pipeline

/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_name = "ruslanmv/ai-medical-chatbot"
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=42).select(range(1000))

In [15]:
with open('hf_token.key', 'r') as f:
    hf_token = f.read()

base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
new_model = "llama-3-8b-counsel"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(base_model, padding='max_length', truncation=True, token = hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.model_max_length = 256

/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def format_chat_template(row):
    row_json = [
        {"role" : "user", "content": row['Patient']},
        {"role" : "assistant", "content": row['Doctor']}
    ]
    
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

In [6]:
dataset = dataset.map(
            format_chat_template,
            num_proc=4
        )

dataset = dataset.train_test_split(test_size=0.1)

In [7]:
# QLoRA Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

In [8]:
# Load Model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config = bnb_config,
    torch_dtype = torch.bfloat16,
    device_map={'':torch.cuda.current_device()}
)

model.config.use_cache=False

Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.98s/it]


In [11]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type = "CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj'],
)

model = get_peft_model(model, peft_config)

In [14]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    overwrite_output_dir=True,
    bf16=True,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=0.1,
    learning_rate=2e-4,
    logging_steps=5,
    logging_strategy="steps",
    log_level="info",
    save_strategy="epoch",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    warmup_steps=10,
    group_by_length=True,
    report_to="none",
    seed=42
)

In [15]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field="text",
    peft_config=peft_config,
    max_seq_length=tokenizer.model_max_length,
    packing= False
)

Map: 100%|██████████| 100/100 [00:00<00:00, 1657.19 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Using auto half precision backend


In [16]:
trainer.train()

***** Running training *****
  Num examples = 900
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Training with DataParallel so batch size has been adjusted to: 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 56
  Number of trainable parameters = 41,943,040
/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
6,3.442000,3.087730
12,2.996200,2.791177
18,2.814600,2.695391
24,2.721400,2.645553
30,2.716200,2.601442
36,2.555400,2.574760
42,2.567200,2.549273
48,2.565600,2.534747
54,2.552400,2.525362


***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to llama-3-8b-counsel/tmp-checkpoint-56
loading configuration file config.json from cache at /data/mn27889/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e1945c40cd546c78e41f1151f4db032b271faeaa/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_d

TrainOutput(global_step=56, training_loss=2.732680755002158, metrics={'train_runtime': 814.3288, 'train_samples_per_second': 1.105, 'train_steps_per_second': 0.069, 'total_flos': 8476279564468224.0, 'train_loss': 2.732680755002158, 'epoch': 0.99})

### Saving the trained model

In [17]:
trainer.model.save_pretrained(new_model)

/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /data/mn27889/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e1945c40cd546c78e41f1151f4db032b271faeaa/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "r

### Merging the base model with the adapter to get full model

In [12]:
base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
new_model = "llama-3-8b-counsel"

In [13]:
# Load Model
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    low_cpu_mem_usage=True,    
    trust_remote_code=True,
    torch_dtype = torch.bfloat16,
    device_map={"":torch.cuda.current_device()}
)

/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:14<00:00,  3.62s/it]


In [16]:
tokenizer = AutoTokenizer.from_pretrained(base_model, padding='max_length', truncation=True, token = hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.model_max_length = 256

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Merge adapter with the base model

In [17]:
model = PeftModel.from_pretrained(base_model_reload, new_model)

In [18]:
model = model.merge_and_unload()

In [19]:
model.save_pretrained("llama-3-8b-chat-doctor")
tokenizer.save_pretrained("llama-3-8b-chat-doctor")

('llama-3-8b-chat-doctor/tokenizer_config.json',
 'llama-3-8b-chat-doctor/special_tokens_map.json',
 'llama-3-8b-chat-doctor/tokenizer.json')

### Load merged Model and Tokenizer for Inference

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    "llama-3-8b-chat-doctor",
    torch_dtype = torch.bfloat16,
    device_map={'':torch.cuda.current_device()}
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.30s/it]


In [8]:
tokenizer = AutoTokenizer.from_pretrained("llama-3-8b-chat-doctor", padding='max_length', truncation=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
tokenizer.model_max_length = 256

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
    tokenizer.convert_tokens_to_ids("<|end_of_text|>")
]

In [10]:
model.config.use_cache = True

messages = [
    {
        "role": "user",
        "content": "Hello doctor, I have acne. How can I get rid of this?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding='max_length', truncation=True).to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    num_return_sequences=1
)

text = tokenizer.decode(outputs[0], skip_special_tokens=False)

print(text.split("assistant")[1])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


<|end_header_id|>

Hi,Thanks for your query. I can understand your concern. Acne is a common skin condition that is caused by the oil glands in the skin getting clogged with dead skin cells and bacteria. Here are some tips to get rid of acne:1. Clean your face twice a day with a mild soap and lukewarm water.2. Use a gentle cleanser that is non-comedogenic (does not clog pores).3. Apply a topical acne cream or gel that contains benzoyl peroxide, salicylic acid, or tea tree oil.4. Avoid touching your face or popping pimples as this can cause further inflammation.5. Avoid using heavy makeup or oily products on your skin.6. Eat a healthy diet that is rich in fruits, vegetables, whole grains, and lean proteins.7. Stay hydrated by drinking plenty of water.8. Avoid stress and anxiety as stress can cause hormonal imbalance and lead to acne.9. Use a warm compress on the affected area for 5-10 minutes to help bring the pus to a head and drain it out.10. Consult a dermatologist if your acne is se